# Options Trading Tool: Machine Learning Exercises
* **Notebook 1: Data Aggregation** – Collects & preprocesses stock & options data  
* **Notebook 2: Hypothesis Testing** – Tests profitability patterns & market behaviors  
* **Notebook 3: Statistical Analysis** – Summarizes results & evaluates statistical significance  
* **Notebook 4: Summary & Insights** – Identifies actionable trading patterns

# Date Aggregation
* Library Imports
* Parameters
* Helper Functions
* Downloading Underlying Data
* Determining Option Contracts
* Downloading Option Data
* Reading Data from CSVs
* Preprocessing Data

In [ ]:
# Import necessary libraries
import os
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import talib as ta
import math

In [ ]:
# API and Date Parameters
api_key = 'Enter API KEY'
api_secret = 'Enter API Secret'

# Today's date (really prior date to avoid conflicts)
today_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

# Symbol and Date Range
symbol = 'SPY'
start_date = "2024-01-01"
# end_date = today_date
end_date = "2024-12-31"
max_calls_per_minute = 150  # For rate limiting purposes
base_folder = 'Historical OHCL Bars'  # Folder to save data
strike_threshold = 0  # Strikes around underlying price to consider
days_offset = 2  # Number of additional days in the future to consider for options

## Helper Functions

In [ ]:
# Helper to read CSVs from a folder
def read_all_csvs(base_folder='Historical OHCL Bars'):
    paths = {
        'options_full_day': os.path.join(base_folder, 'Options'),
        'underlying': os.path.join(base_folder, 'Underlying')
    }

    def read_and_concat_files(folder):
        if os.path.exists(folder):
            files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.csv')]
            if files:
                dfs = [pd.read_csv(f) for f in files if os.stat(f).st_size > 0]
                if dfs:
                    return pd.concat(dfs, ignore_index=True)
        return pd.DataFrame()

    data = {
        'options_full_day': read_and_concat_files(paths['options_full_day']),
        'underlying': read_and_concat_files(paths['underlying'])
    }
    
    return data

In [ ]:
# Helper to save CSV data
def save_to_csv(data, date_str, session, folder, suffix):
    os.makedirs(folder, exist_ok=True)
    file_path = os.path.join(folder, f'{suffix}_{date_str}.csv')
    data.to_csv(file_path, index=False)
    print(f"Data saved to {file_path}")

In [ ]:
# Helper to generate option symbol
def generate_option_symbol(symbol, exp_date, call_put, strike_price_dollar, strike_price_decimal):
    strike_price = f"{int(strike_price_dollar):05d}{int(strike_price_decimal):03d}"
    exp_date_formatted = datetime.strptime(exp_date, "%Y-%m-%d").strftime("%y%m%d")
    full_symbol = f"{symbol}{exp_date_formatted}{call_put}{strike_price}"
    return full_symbol

In [ ]:
def calculate_strike_ranges(df, strike_threshold):
    df['t'] = pd.to_datetime(df['timestamp'])  # Convert timestamp to datetime
    df['date'] = df['t'].dt.date
    # Use 'high' and 'low' based on your CSV structure
    daily_ranges = df.groupby('date').agg({'low': 'min', 'high': 'max'})

    strike_ranges = {}
    for date, row in daily_ranges.iterrows():
        low_end = row['low'] // 1
        high_end = row['high'] // 1
        low_strike = int(low_end) - strike_threshold
        high_strike = int(high_end) + strike_threshold
        strike_ranges[date] = (low_strike, high_strike)
    
    return strike_ranges


In [ ]:
def generate_contracts_to_consider(df, low_strike, high_strike, expiration_dates):
    """
    Generate unique contracts to consider for a specific day.
    Contracts will be unique within a day but may repeat across different days.
    """
    contracts = set()  # Use a set to ensure uniqueness within the day

    # Iterate through the underlying data for the day
    for _, row in df.iterrows():
        for strike in range(int(low_strike), int(high_strike) + 1):
            strike_price_dollar = int(strike)
            strike_price_decimal = 0
            for exp_date in expiration_dates:
                call_contract = generate_option_symbol(row['symbol'], exp_date, 'C', strike_price_dollar, strike_price_decimal)
                put_contract = generate_option_symbol(row['symbol'], exp_date, 'P', strike_price_dollar, strike_price_decimal)
                
                # Add contracts to the set (automatically handles uniqueness)
                contracts.add(call_contract)
                contracts.add(put_contract)

    return list(contracts)  # Convert the set back to a list to return


In [ ]:
# Helper to get expiration dates
def get_expiration_dates(date_str, include_today=True, offset_days=2):
    base_date = np.datetime64(datetime.strptime(date_str, "%Y-%m-%d"), 'D')
    holidays = ['2024-01-01', '2024-07-04', '2024-12-25']
    holidays_np = np.array(holidays, dtype='datetime64[D]')
    start_day_offset = 0 if include_today else 1
    expiration_dates = [
        np.busday_offset(base_date, i, roll='forward', holidays=holidays_np).astype(str)
        for i in range(start_day_offset, start_day_offset + offset_days)
    ]
    return [datetime.strptime(date, '%Y-%m-%d').strftime('%Y-%m-%d') for date in expiration_dates]

In [ ]:
def save_options_ohlc_to_csv(df, date_str, base_folder='Historical OHCL Bars'):
    """
    Saves the consolidated options OHLC data to a CSV file for a given date.
    
    Parameters:
        df (DataFrame): The consolidated options data.
        date_str (str): The date for which data is being saved.
        base_folder (str): The folder where the file will be saved.
    """
    # Define the file path for saving the data
    options_folder = os.path.join(base_folder, 'Options')
    os.makedirs(options_folder, exist_ok=True)
    file_path = os.path.join(options_folder, f'options_{date_str}.csv')

    # Save the DataFrame to CSV
    df.to_csv(file_path, index=False)
    print(f"Data saved to {file_path}")

In [ ]:
# Modify the save_underlying_ohlc_to_csv function to clean up the DataFrame
def save_underlying_ohlc_to_csv(stock_bars_data, date_str, symbol, base_folder='Historical OHCL Bars'):
    """Save underlying stock bars data to a CSV file."""
    underlying_folder = os.path.join(base_folder, 'Underlying')
    os.makedirs(underlying_folder, exist_ok=True)

    # Extract bars data from the response
    bars = stock_bars_data['bars']

    # Convert the bars data to a DataFrame
    df = pd.DataFrame(bars)

    # Add the symbol to the DataFrame
    df['symbol'] = symbol

    # Clean up the column names to be more intuitive
    df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'volume', 't': 'timestamp', 'vw': 'vwap'}, inplace=True)

    # Convert timestamp to a readable datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Define file path with date stamp
    file_path = os.path.join(underlying_folder, f'underlying_{symbol}_{date_str}.csv')

    # Save the DataFrame to CSV
    df.to_csv(file_path, index=False)
    print(f"Underlying data for {symbol} on {date_str} saved successfully to {file_path}!")


## Download Underlying Data

In [ ]:
# Helper function to fetch stock bars (OHLC) from Alpaca API
def get_stock_bars(symbol, timeframe, start_date, end_date, api_key, api_secret, limit=10000, adjustment='raw', feed='sip', sort='asc'):
    """
    Fetch OHLC data from Alpaca API for a given stock symbol.

    Parameters:
        symbol (str): Stock symbol (e.g., "SPY")
        timeframe (str): Timeframe for the bars (e.g., "1Min", "5Min", etc.)
        start_date (str): Start date for the data (ISO 8601 format with 'T' and 'Z' time)
        end_date (str): End date for the data (ISO 8601 format with 'T' and 'Z' time)
        api_key (str): Alpaca API key
        api_secret (str): Alpaca API secret
        limit (int): Maximum number of bars to fetch (default is 10000)
        adjustment (str): Data adjustment (default is 'raw')
        feed (str): Data feed (default is 'sip')
        sort (str): Sorting order ('asc' or 'desc', default is 'asc')

    Returns:
        dict: JSON response containing OHLC data
    """
    url = (
        f"https://data.alpaca.markets/v2/stocks/{symbol}/bars?timeframe={timeframe}"
        f"&start={start_date}&end={end_date}&limit={limit}&adjustment={adjustment}&feed={feed}&sort={sort}"
    )

    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": api_secret
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Error fetching stock bars: {response.status_code}, {response.text}")

    # Print the response to check the structure
    print(f"Response JSON: {response.json()}")

    return response.json()



In [ ]:
def fetch_full_day_intraday_data(symbol, start_date, end_date, api_key, api_secret, max_calls_per_minute):
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')  # Only business days
    call_count = 0

    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        print(f"Fetching data for {date_str}")

        # Modify this part to match the correct parameters expected by get_stock_bars
        stock_bars_data = get_stock_bars(
            symbol=symbol,
            timeframe="1Min",  # 1Min OHLC data
            start_date=date_str,  # Adjust to match the expected format
            end_date=date_str,  # Adjust to match the expected format
            api_key=api_key,
            api_secret=api_secret
        )

        # Check if there is data in the response
        if stock_bars_data.get('bars') is None:
            print(f"No data found for {date_str}. Skipping...")
            continue

        # Save data to CSV
        save_underlying_ohlc_to_csv(stock_bars_data, date_str, symbol)

        # Update call count and implement rate limiting
        call_count += 1
        if call_count >= max_calls_per_minute:
            print("Rate limiting... waiting for 1 minute.")
            time.sleep(60)  # Pause for 60 seconds
            call_count = 0  # Reset the call count after waiting


## Determine Option Contracts Based On Underlying Data

In [ ]:
# Process and generate unique contracts for each day based on underlying data
def process_and_generate_all_contracts(df_underlying, strike_threshold=0, offset_days=2):
    strike_ranges = calculate_strike_ranges(df_underlying, strike_threshold)
    all_contracts = {}

    for date_obj in sorted(df_underlying['date'].unique()):
        date_str = date_obj.strftime("%Y-%m-%d")
        if date_obj in strike_ranges:
            low_strike, high_strike = strike_ranges[date_obj]
            expiration_dates = get_expiration_dates(date_str, include_today=True, offset_days=offset_days)
            contracts = generate_contracts_to_consider(df_underlying[df_underlying['date'] == date_obj], low_strike, high_strike, expiration_dates)
            
            # Ensure contracts are unique for each date
            if date_str not in all_contracts:
                all_contracts[date_str] = contracts
            else:
                all_contracts[date_str].extend(contracts)
                all_contracts[date_str] = list(set(all_contracts[date_str]))  # Ensure uniqueness
        else:
            print(f"No data available for {date_str}")
    
    return all_contracts

## Download Option Data

In [ ]:
def fetch_full_day_option_ohlc(contracts_by_date, api_key, api_secret, max_calls_per_minute=200):
    """
    Fetch OHLC data for options contracts for each date, adhering to API call limits, and save into one file per day.

    Parameters:
        contracts_by_date (dict): A dictionary with dates as keys and lists of contracts as values.
        api_key (str): Your Alpaca API key.
        api_secret (str): Your Alpaca API secret.
        max_calls_per_minute (int): Limit on the number of API calls per minute.
    """
    call_count = 0
    start_time = time.time()  # Track start time to enforce rate limit

    for date_str, symbols in contracts_by_date.items():
        print(f"Fetching option data for {date_str}")

        consolidated_data = []  # Store all data for this day
        
        for symbol in symbols:
            print(f"Fetching data for symbol {symbol} on {date_str}")
            
            # Fetch 1-minute OHLC data for the full day using the fetch_option_bars function
            option_bars_data = fetch_option_bars(
                symbols=[symbol],  # Ensure symbol is passed as a list
                timeframe="1Min",  # 1Min OHLC data
                # start=f"{date_str}T09:30:00Z",
                # end=f"{date_str}T16:00:00Z",
                start=f"{date_str}",
                end=f"{date_str}",
                api_key=api_key,
                api_secret=api_secret
            )

            # Increment the API call counter
            call_count += 1

            # If there is no data, skip this symbol
            if not option_bars_data or 'bars' not in option_bars_data:
                print(f"No data to save for {symbol} on {date_str}.")
                continue

            # Convert the option bars data to a list of dictionaries
            for symbol_key, bars in option_bars_data['bars'].items():
                for bar in bars:
                    bar['symbol'] = symbol_key
                    consolidated_data.append(bar)

            # Rate limiting: if we've hit the max calls per minute, pause
            if call_count >= max_calls_per_minute:
                elapsed_time = time.time() - start_time
                if elapsed_time < 60:  # If less than a minute has passed, wait
                    sleep_time = 60 - elapsed_time
                    print(f"Rate limit reached. Sleeping for {sleep_time:.2f} seconds.")
                    time.sleep(sleep_time)
                
                # Reset the counter and start time after pausing
                call_count = 0
                start_time = time.time()

        # Convert the consolidated data into a DataFrame
        if consolidated_data:
            df = pd.DataFrame(consolidated_data)
            # Convert timestamp to readable datetime format
            df['t'] = pd.to_datetime(df['t'])

            # Save the consolidated data for the day
            save_options_ohlc_to_csv(df, date_str)

        else:
            print(f"No data to save for {date_str}.")

In [ ]:
def fetch_option_bars(symbols, timeframe="5Min", start=None, end=None, limit=1000, sort="asc", 
                      api_key=None, api_secret=None):
    """
    Fetches option bars data for given option symbols from Alpaca Broker.

    Parameters:
        symbols (list or str): A list of option symbols or a single option symbol (e.g., ["AAPL241220C00300000", "AAPL240315C00225000"]).
        timeframe (str): The timeframe for the bars (default is "5Min").
        start (str): The start date in "YYYY-MM-DD" format.
        end (str): The end date in "YYYY-MM-DD" format.
        limit (int): The maximum number of bars to fetch (default is 1000).
        sort (str): The sort order, either "asc" or "desc" (default is "asc").
        api_key (str): Your Alpaca API key.
        api_secret (str): Your Alpaca API secret.

    Returns:
        dict: The response from the Alpaca API in JSON format.
    """
    # Ensure symbols are properly formatted as a comma-separated string
    if isinstance(symbols, list):
        symbols = ",".join(symbols)

    url = f"https://data.alpaca.markets/v1beta1/options/bars?symbols={symbols}&timeframe={timeframe}&limit={limit}&sort={sort}"
    
    if start:
        url += f"&start={start}"
    if end:
        url += f"&end={end}"

    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": api_key,
        "APCA-API-SECRET-KEY": api_secret
    }

    response = requests.get(url, headers=headers)
    
    # Check if the response is successful
    if response.status_code != 200:
        print(f"Error fetching option bars: {response.status_code}, {response.text}")
        return {}
    
    # Parse response JSON
    response_data = response.json()

    # Debugging: print the full response from the API
    print(f"API response for symbols {symbols}: {response_data}")

    # Check if there is any data in 'bars'
    if 'bars' not in response_data or not response_data['bars']:
        print(f"No bars data returned for symbols: {symbols}")
        return {}

    return response_data


## Preprocessing
* merging
* technical indicators
* moneyness: ITM, ATM, OTM
* greeks

In [ ]:
def add_underlying_price_and_moneyness(df_options, df_underlying):
    # Ensure both DataFrames have compatible datetime formats and are tz-naive
    df_options['t'] = pd.to_datetime(df_options['t']).dt.tz_localize(None)
    df_underlying['t'] = pd.to_datetime(df_underlying['t']).dt.tz_localize(None)

    # Print time ranges to check for overlap
    print(f"Options time range: {df_options['t'].min()} to {df_options['t'].max()}")
    print(f"Underlying time range: {df_underlying['t'].min()} to {df_underlying['t'].max()}")

    # Trim the underlying data to ensure the time range aligns with options data
    df_underlying_trimmed = df_underlying[df_underlying['t'] >= df_options['t'].min()]

    # Print size of underlying after trimming
    print(f"Size of underlying after trimming: {df_underlying_trimmed.shape}")

    # Merge using the 'close' column from df_underlying, but ensure we are merging on the same minute-level timestamps
    df = pd.merge_asof(
        df_options.sort_values('t'),
        df_underlying_trimmed[['t', 'close']].sort_values('t'),
        on='t',
        direction='backward',  # Find the last available price
        tolerance=pd.Timedelta('1min')  # Match within 1-minute tolerance
    )

    # Print size of resulting DataFrame
    print(f"Size of merged DataFrame: {df.shape}")
    
    # If the merged DataFrame is empty, return and stop further processing
    if df.empty:
        print("Warning: The merged DataFrame is empty!")
        return df

    # Extract the last 6 digits from the symbol and divide by 1000 to get the strike price
    df['strike_price'] = df['symbol'].str[-6:].astype(float) / 1000

    # Define a function to label moneyness based on the option type and underlying close price
    def label_moneyness(row):
        if 'C' in row['symbol']:  # For Call options
            if row['close'] > row['strike_price']:
                return 'ITM'
            elif row['close'] == row['strike_price']:
                return 'ATM'
            else:
                return 'OTM'
        elif 'P' in row['symbol']:  # For Put options
            if row['close'] < row['strike_price']:
                return 'ITM'
            elif row['close'] == row['strike_price']:
                return 'ATM'
            else:
                return 'OTM'
        return None

    # Apply moneyness labeling
    df['moneyness'] = df.apply(label_moneyness, axis=1)

    return df


In [ ]:
def add_technical_indicators(df):
    df['RSI'] = ta.RSI(df['c'], timeperiod=14)
    df['MACD'], df['MACD_signal'], df['MACD_hist'] = ta.MACD(df['c'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['BB_upper'], df['BB_middle'], df['BB_lower'] = ta.BBANDS(df['c'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
    df['SMA'] = ta.SMA(df['c'], timeperiod=20)
    df['EMA'] = ta.EMA(df['c'], timeperiod=20)
    return df

In [ ]:
def preprocess_full_pipeline(df_options, df_underlying):
    df_options = add_technical_indicators(df_options)
    print(f"After adding technical indicators: {df_options.shape}")
    
    df_options = add_underlying_price_and_moneyness(df_options, df_underlying)
    print(f"After adding underlying price and moneyness: {df_options.shape}")
    
#     df_options = add_greeks(df_options)
#     print(f"After adding Greeks: {df_options.shape}")
    
    return df_options


## Actuals
* Get underlying data
* Get Option data
* Preprocess

In [ ]:
# Step 1: Download underlying data
fetch_full_day_intraday_data(
    symbol=symbol,
    start_date=start_date,
    end_date=end_date,
    api_key=api_key,
    api_secret=api_secret,
    max_calls_per_minute=max_calls_per_minute
)

In [ ]:
# Step 2: Determine contracts to download
data = read_all_csvs(base_folder)
df_underlying = data['underlying']
all_contracts = process_and_generate_all_contracts(df_underlying, strike_threshold=strike_threshold, offset_days=days_offset)

In [ ]:
df_underlying.head()

In [ ]:
# size of all contracts

def count_contracts_by_date(all_contracts):
    """
    Returns the count of option contracts for each date.

    Parameters:
        all_contracts (dict): A dictionary with dates as keys and lists of contracts as values.

    Returns:
        dict: A dictionary with dates as keys and the count of contracts as values.
    """
    contract_counts = {}
    
    for date, contracts in all_contracts.items():
        contract_counts[date] = len(contracts)
    
    return contract_counts

count_contracts_by_date(all_contracts)

In [ ]:
# Step 3: Download option data
contracts_by_date = {date: contracts for date, contracts in all_contracts.items()}

fetch_full_day_option_ohlc(
    contracts_by_date=contracts_by_date,
    api_key=api_key,
    api_secret=api_secret,
    max_calls_per_minute=max_calls_per_minute
)

In [ ]:
print(df_underlying.columns)

In [ ]:
df_underlying.head()

In [ ]:
# Step 4: Read saved data and preprocess
data = read_all_csvs(base_folder)
df_options_full_day = data['options_full_day']
df_processed = preprocess_full_pipeline(df_options_full_day, df_underlying)

In [ ]:
# Step 5: Display processed data
print(df_processed.head(15))

In [ ]:
df_processed.tail()

In [ ]:
# Define the path where you want to save the file

save_path_underlying = r"ENTER FILE PATH"
save_path_options = r"ENTER FILE PATH"

# Save the DataFrame to the specified path as a CSV file
df_underlying.to_csv(save_path_underlying, index=False)
df_processed.to_csv(save_path_options, index=False)

print(f"DataFrame saved to {save_path_underlying}")
print(f"DataFrame saved to {save_path_options}")